<a href="https://colab.research.google.com/github/LaAbadIAdelCrimen/abadia-3d/blob/master/abadia_3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
import pandas as pd
from sys import exit


# Reading the data from a CSV file using pandas
repo = pd.read_csv('alturas-extra.csv',sep=',',header=0)
# print(repo)


data = np.array((repo['X'].values, repo['Y'].values, repo['height'].values, repo['Level'].values))
print(data.shape[1])
# Create heatmap

data0 = np.asarray([x for x in data if x[3] == 0 ])
print(data0.shape)

heatmap, xedges, yedges = np.histogram2d(data[1], data[0], bins=(256,256))
extent = [xedges[0], xedges[256], yedges[0], yedges[256]]

# Plot heatmap
plt.clf()
plt.title('Plano de Alturas de la Abadia del Crimen')
plt.ylabel('x')
plt.xlabel('y')
plt.imshow(heatmap, extent=extent)
plt.show()

In [0]:
maps = np.zeros((3, 256, 256))
for dd in repo.values:
  # print (dd)
  maps[dd[0], dd[2], dd[1]] = dd[3]
  # print (maps[dd[0], dd[1], dd[2]])

# print (maps[2])

fig  = plt.figure(figsize=(10,10),facecolor = 'white', dpi=100)
plt.savefig('abadia-3d-planta0.png', dpi=100)

plt.imshow (maps[0], cmap='Blues', interpolation='none')
plt.show(fig)

fig  = plt.figure(figsize=(10,10),facecolor = 'white', dpi=100)
plt.savefig('abadia-3d-planta1.png', dpi=100)

plt.imshow (maps[1], cmap='Blues', interpolation='none')
plt.show(fig)

fig  = plt.figure(figsize=(10,10),facecolor = 'white', dpi=100)
plt.savefig('abadia-3d-planta2.png', dpi=100)

plt.imshow (maps[2], cmap='Blues', interpolation='none')
plt.show(fig)
